In [1]:
# Reference : https://github.com/golbin/TensorFlow-Tutorials 
import tensorflow as tf
import numpy as np

- S: 디코딩 입력의 시작을 나타내는 심볼
- E: 디코딩 출력의 끝을 나타내는 심볼
- P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼

```
예) 현재 배치 데이터의 최대 크기가 4인 경우
word -> ['w', 'o', 'r', 'd']
to   -> ['t', 'o', 'P', 'P']
```

In [2]:
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

In [3]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        # 인코더 셀의 입력값
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값. 시작을 타나내는 S 심볼을 맨 앞에 붙여준다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E를 붙여준다.
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        
        # sparse_softmax_cross_entropy_with_logits 사용할거라 one-hot 인코딩이 아님
        target_batch.append(target)
        
    return input_batch, output_batch, target_batch

In [4]:
# 파라미터 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

# 입력과 출력의 형태가 one-hot 인코딩으로 같기 때문에
n_class = n_input = dic_len

In [5]:
# 모델 구성

# [batch_size, time_steps, input_size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

# 인코더 셀 구성
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)
    # outputs: [batch_size, step_size, n_hidden]
    # enc_states: [batch_size, n_hidden(=128)]
    
# 디코더 셀 구성
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    
    # seq2seq에서는 인코더 셀의 최종 상태값을 디코더 셀의 초기값으로 넣어줘야함
    outputs, enc_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)
    
    
    
model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
outputs : Tensor("encode/rnn/transpose_1:0", shape=(?, ?, 128), dtype=float32)
Instructions for updating:
Use keras.layers.dense instead.


In [6]:
# 신경망 모델 학습

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={
                                                     enc_input: input_batch, 
                                                     dec_input: output_batch,
                                                     targets: target_batch,
                                                    })
    
    print('Epoch :', '%04d' % (epoch + 1), 'cost = ', '{:.6f}'.format(loss))

Epoch : 0001 cost =  3.765495
Epoch : 0002 cost =  2.848657
Epoch : 0003 cost =  1.686130
Epoch : 0004 cost =  1.003551
Epoch : 0005 cost =  0.609220
Epoch : 0006 cost =  0.342718
Epoch : 0007 cost =  0.290152
Epoch : 0008 cost =  0.255709
Epoch : 0009 cost =  0.265073
Epoch : 0010 cost =  0.122246
Epoch : 0011 cost =  0.083720
Epoch : 0012 cost =  0.153749
Epoch : 0013 cost =  0.204028
Epoch : 0014 cost =  0.116245
Epoch : 0015 cost =  0.160800
Epoch : 0016 cost =  0.123206
Epoch : 0017 cost =  0.084600
Epoch : 0018 cost =  0.050823
Epoch : 0019 cost =  0.091640
Epoch : 0020 cost =  0.048518
Epoch : 0021 cost =  0.059746
Epoch : 0022 cost =  0.174631
Epoch : 0023 cost =  0.013302
Epoch : 0024 cost =  0.024974
Epoch : 0025 cost =  0.237662
Epoch : 0026 cost =  0.020074
Epoch : 0027 cost =  0.260382
Epoch : 0028 cost =  0.047037
Epoch : 0029 cost =  0.008343
Epoch : 0030 cost =  0.010418
Epoch : 0031 cost =  0.059241
Epoch : 0032 cost =  0.114934
Epoch : 0033 cost =  0.023340
Epoch : 00

In [7]:
# 번역 테스트
def translate(word):
    # 예측시 디코더의 입출력값을 의미 없는 값인 P로 채운다.
    seq_data = [word, 'P' * len(word)]
    
    input_batch, output_batch, target_batch = make_batch([seq_data])
    
    # 결과가 [batch_size, time_step, input] 으로 나오기 때문에
    # 2번째 차원원인 input에서 가장 확률이 높은 글자를 예측값으로 만든다.
    prediction = tf.argmax(model, 2)
    
    result = sess.run(prediction, feed_dict={
        enc_input: input_batch,
        dec_input: output_batch,
        targets: target_batch,
    })
    
    decoded = [char_arr[i] for i in result[0]]
    
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    print(decoded)
    
    return translated

print('\n=== 번역 테스트 ===')

print('word ->', translate('wordword'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
['소', '녀', '사', 'r', '스', '이', 'E', '나', '소']
word -> 소녀사r스이
['나', '무', 'E', '녀', 'E']
wodr -> 나무
['사', '랑', 'E', 'o', '소']
love -> 사랑
['사', '랑', 'E', '이', '소']
loev -> 사랑
['키', '스', 'E', '이', '소']
abcd -> 키스
